In [1]:
import numpy
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import gzip
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
    


In [2]:
user_id_list = []
items_count_list =[]
steam_id_list =[]
url_list =[]
items_list =[]
for l in  readGz('australian_users_items.json.gz'):
    user_id,items_count,steam_id,url,items = l['user_id'],l['items_count'],l['steam_id'],l['user_url'],l['items']
    user_id_list.append(user_id)
    items_count_list.append(items_count)
    steam_id_list.append(steam_id)
    url_list.append(url)
    items_list.append(items)

In [2]:
item_id_name= defaultdict(list)
for l in range(len(items_list)):
    for i in range(len(items_list[l])):
        item_id,item_name = items_list[l][i]['item_id'],items_list[l][i]['item_name']
        item_id_name[item_id].append(item_name)
id_name_set ={}
for l in item_id_name:
    id_name_set[l] =set(item_id_name[l])


In [16]:
import random
item_recommend = defaultdict(list)
item_review = defaultdict(list)
review_helpful =defaultdict(list)
review_recommend = defaultdict(list)
text = []
data2 = list(readGz('australian_user_reviews.json.gz'))

random.shuffle(data2)
length = len(data2)
review_train = data2[0:round(length*0.5)]
review_validation =data2[round(length*0.5):round(length*0.8)]
review_test  = data2 [round(length*0.8):]
for l in  review_train:
    review = l['reviews']
    text.append(review)
    for i in review:
        funny,posted,item_id,helpful,recommend,review_text =i['funny'],i['posted'],i['item_id'],i['helpful'],i['recommend'],i['review']
        review_recommend[review_text].append(recommend)
        review_helpful[review_text].append(helpful)
        item_review[item_id].append(review_text)
        item_recommend[item_id].append(recommend)

In [4]:

review_list =[]
for i in range(len(text)):
    for l in text[i]:
        review_list.append(l['review'])
for i in review_recommend:
    if len(review_recommend[i]) >1:
        review_recommend[i]=review_recommend[i][0]

In [5]:
 
punctuation = set(string.punctuation)
old_w = ''
cob=''
wordCount = defaultdict(int)
for l in review_recommend:
    l =str(l)
    r = ''.join([c for c in l.lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

In [6]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]

In [7]:
idf_set =defaultdict(int)
count = 0
indicator = 0
for i in words:
    for j in review_recommend:
        j = str(j)
        if i in set(j.split()):
            count+= 1
    idf_set[i] = count
    indicator+=1
    count=0

In [8]:
import math
n =len(review_recommend)
for i in idf_set:
    idf_set[i]=math.log10(n/(idf_set[i]+1))
tf_set_r = defaultdict(int)


In [9]:
                          
for i in range(0,len(review_recommend)):
    tf_set_r[i]=defaultdict(int)
    for k in idf_set:
        tf_set_r[i][k]=0


In [10]:
i=0
for k in review_recommend:
    k = str(k)
    r = ''.join([c for c in k.lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set:
            if j == w:
                tf_set_r[i][j]+=1
    i+=1

In [11]:
tf_idf_r = defaultdict(int)
for i in range(0,len(review_recommend)):
    tf_idf_r[i] =[]
for k in range(len(tf_set_r)):
    for i in idf_set:
        tf_idf_r[k].append(tf_set_r[k][i]*idf_set[i])

In [12]:
recommend_list=[]
for i in review_recommend:
    recommend_list.append(review_recommend[i])

In [13]:
for i in range(len(recommend_list)):
    if recommend_list[i] == [True]:
        recommend_list[i] =True
    elif recommend_list[i]==[False]:
        recommend_list[i]= False

In [22]:
temp_list_train=[]
new_list_train = []
a= 0
for i in range(len(tf_idf_r)):
        temp_list_train.append(1)
        for j in range(len(tf_idf_r[i])):
            temp_list_train.append(tf_idf_r[i][j])
        new_list_train.append(temp_list_train)
        temp_list_train=[]
X_train = new_list_train
y_train = recommend_list

In [15]:
from sklearn import linear_model
regularization =[0.01,0.1,1,10,100]
for k in regularization : 
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_train)
    s= 0
    for i in range(len(y_train)):
        s +=(y_train[i]-predictions[i])**2
    MSE = s/len(y_train)
    print(k,MSE)

0.01 0.07797986111547041
0.1 0.07797986227201174
1 0.07797996427670166
10 0.07798428647535584
100 0.07808515013399189


In [16]:
dict_word ={}
a = 0
for i in range(1,1001):
    dict_word[words[i-1]] = theta[i]

In [17]:
counts1 = [(dict_word[w], w) for w in dict_word]
counts1.sort()
counts1.reverse()

In [18]:
data3 = list(readGz('australian_user_reviews.json.gz'))
item_review_whole=defaultdict(list)
for l in data3:
    review = l['reviews']
    for i in review:
        item_id,review_text = i['item_id'],i['review']
        item_review_whole[item_id].append(review_text)

In [19]:
item_work = {}
for i in item_review_whole:
    item_work[i]=set()
for i in item_review_whole:
    for j in range(len(item_review_whole[i])):
        l =str(item_review_whole[i][j])
        r = ''.join([c for c in l.lower() if not c in punctuation])
        for w in r.split():
            item_work[i].add(w)

In [20]:
item_work_score = {}
for i in item_work:
    item_work_score[i]={}
for i in item_work:
    for j in item_work[i]:
        if j in dict_word:
            item_work_score[i][j]=dict_word[j]
        else:
            item_work_score[i][j]= 0

In [21]:
top5_dict = {}
for i in item_work_score:
    sort_d = []
    sort_d=sorted(item_work_score[i].items(),key = lambda d:d[1],reverse=True)
    top5_dict[i]=sort_d[0:5]

In [8]:
    review_helpful_train= defaultdict(list)
    review_helpful_test = defaultdict(list)
    review_text = []
    data4 = list(readGz('australian_user_reviews.json.gz'))


    

In [17]:
    a=[]
    punctuation = set(string.punctuation)
    
    for l in  data4:
        review= l['reviews']
        for i in review:
            helpful,review_important,item =i['helpful'],i['review'],i['item']
            if helpful != 'No ratings yet':
                k =str(helpful)
                r = ''.join([c for c in k.lower() if not c in punctuation])
                if int(a[2])>=5:
                a= r.split()
                review_helpful_train[review_important].append(helpful)
            else:
                review_helpful_test[review_important].append(helpful)
            


In [18]:
len(review_helpful_test)

58995

In [127]:
helpful_set_test =defaultdict(list)
for i in review_helpful_test:
    helpful_set_test[i].append(set(review_helpful_test[i]))


In [43]:
helpful_set_train= defaultdict(list)
helpful_set_validation= defaultdict(list)
length =len(review_helpful_train)/2
a = 0
for i in review_helpful_train:
    if a < length:
        helpful_set_train[i].append(set(review_helpful_train[i]))
    else:
        helpful_set_validation[i].append(set(review_helpful_train[i]))
    a+=1

In [44]:
punctuation = set(string.punctuation)
old_w = ''
cob=''
wordCount = defaultdict(int)
for l in helpful_set_train:
    l =str(l)
    r = ''.join([c for c in l.lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1
        
counts5 = [(wordCount[w], w) for w in wordCount]
counts5.sort()
counts5.reverse()
words_helpful = [x[1] for x in counts5[:2000]]        
        
        
idf_set_helpful =defaultdict(int)
count = 0
for i in words_helpful:
    for j in helpful_set_train:
        j = str(j)
        if i in set(j.split()):
            count+= 1
    idf_set_helpful[i] = count
    count=0

In [128]:
import math
idf_set_helpful_test =defaultdict(int)
count = 0
for i in words_helpful:
    for j in helpful_set_test:
        j = str(j)
        if i in set(j.split()):
            count+= 1
    idf_set_helpful_test[i] = count
    count=0
n =len(helpful_set_test)
for i in idf_set_helpful_test:
    idf_set_helpful_test[i]=math.log10(n/(idf_set_helpful_test[i]+1))
tf_set_r_helpful_test = defaultdict(int)
for i in range(0,len(helpful_set_test)):
    tf_set_r_helpful_test[i]=defaultdict(int)
    for k in idf_set_helpful_test:
        tf_set_r_helpful_test[i][k]=0
i=0
for k in helpful_set_test:
    k = str(k)
    r = ''.join([c for c in k.lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set_helpful_test:
            if j == w:
                tf_set_r_helpful_test[i][j]+=1
    i+=1
tf_idf_r_helpful_test = defaultdict(int)
for i in range(0,len(helpful_set_test)):
    tf_idf_r_helpful_test[i] =[]
for k in range(len(tf_set_r_helpful_test)):
    for i in idf_set_helpful_test:
        tf_idf_r_helpful_test[k].append(tf_set_r_helpful_test[k][i]*idf_set_helpful_test[i])

In [129]:

new_helpful_test = []
a= 0
for i in range(len(tf_idf_r_helpful_test)):
        temp_helpful_test.append(1)
        for j in range(len(tf_idf_r_helpful_test[i])):
            temp_helpful_test.append(tf_idf_r_helpful_test[i][j])
        new_helpful_test.append(temp_helpful_test)
        temp_helpful_test=[]
X_test = new_helpful_test

In [45]:

n =len(helpful_set_train)
for i in idf_set_helpful:
    idf_set_helpful[i]=math.log10(n/(idf_set_helpful[i]+1))
tf_set_r_helpful = defaultdict(int)
for i in range(0,len(helpful_set_train)):
    tf_set_r_helpful[i]=defaultdict(int)
    for k in idf_set_helpful:
        tf_set_r_helpful[i][k]=0
i=0
for k in helpful_set_train:
    k = str(k)
    r = ''.join([c for c in k.lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set_helpful:
            if j == w:
                tf_set_r_helpful[i][j]+=1
    i+=1

In [46]:
tf_idf_r_helpful = defaultdict(int)
for i in range(0,len(helpful_set_train)):
    tf_idf_r_helpful[i] =[]
for k in range(len(tf_set_r_helpful)):
    for i in idf_set_helpful:
        tf_idf_r_helpful[k].append(tf_set_r_helpful[k][i]*idf_set_helpful[i])

In [47]:
total_number =[]
helpful_number =[]
a = []
for i in helpful_set_train:
    if len(helpful_set_train[i])== 1:
        l =str(helpful_set_train[i])
        r = ''.join([c for c in l.lower() if not c in punctuation])
        a= r.split()
        helpful_number.append(a[0])
        total_number.append(a[2])
        a = []
    else:
        l =str(helpful_set_train[i][0])
        r = ''.join([c for c in l.lower() if not c in punctuation])
        a= r.split()
        helpful_number.append(a[0])
        total_number.append(a[2])
        a = []
        

In [48]:
helpful_rate_train =[]
for i in range(len(helpful_number)):
    helpful_rate_train.append((int(helpful_number[i]))/(int(total_number[i])))

In [49]:
temp_helpful_train=[]
new_helpful_train = []
a= 0
for i in range(len(tf_idf_r_helpful)):
        temp_helpful_train.append(1)
        for j in range(len(tf_idf_r_helpful[i])):
            temp_helpful_train.append(tf_idf_r_helpful[i][j])
        new_helpful_train.append(temp_helpful_train)
        temp_helpful_train=[]
X_train = new_helpful_train
y_train = helpful_rate_train

In [54]:
import math
idf_set_helpful_validation =defaultdict(int)
count = 0
for i in words_helpful:
    for j in helpful_set_validation:
        j = str(j)
        if i in set(j.split()):
            count+= 1
    idf_set_helpful_validation[i] = count
    count=0
n =len(helpful_set_validation)
for i in idf_set_helpful_validation:
    idf_set_helpful_validation[i]=math.log10(n/(idf_set_helpful_validation[i]+1))
tf_set_r_helpful_validation = defaultdict(int)
for i in range(0,len(helpful_set_validation)):
    tf_set_r_helpful_validation[i]=defaultdict(int)
    for k in idf_set_helpful_validation:
        tf_set_r_helpful_validation[i][k]=0
i=0
for k in helpful_set_validation:
    k = str(k)
    r = ''.join([c for c in k.lower() if not c in punctuation])
    for w in r.split():
        for j in idf_set_helpful_validation:
            if j == w:
                tf_set_r_helpful_validation[i][j]+=1
    i+=1
tf_idf_r_helpful_validation = defaultdict(int)
for i in range(0,len(helpful_set_validation)):
    tf_idf_r_helpful_validation[i] =[]
for k in range(len(tf_set_r_helpful_validation)):
    for i in idf_set_helpful_validation:
        tf_idf_r_helpful_validation[k].append(tf_set_r_helpful_validation[k][i]*idf_set_helpful_validation[i])

In [55]:
total_number_validation =[]
helpful_number_validation =[]
a = []
for i in helpful_set_validation:
    if len(helpful_set_validation[i])== 1:
        l =str(helpful_set_validation[i])
        r = ''.join([c for c in l.lower() if not c in punctuation])
        a= r.split()
        helpful_number_validation.append(a[0])
        total_number_validation.append(a[2])
        a = []
    else:
        l =str(helpful_set_validation[i][0])
        r = ''.join([c for c in l.lower() if not c in punctuation])
        a= r.split()
        helpful_number_validation.append(a[0])
        total_number_validation.append(a[2])
        a = []
helpful_rate_validation =[]
for i in range(len(helpful_number_validation)):
    helpful_rate_validation.append((int(helpful_number_validation[i]))/(int(total_number_validation[i])))
temp_helpful_validation=[]
new_helpful_validation = []
a= 0
for i in range(len(tf_idf_r_helpful_validation)):
        temp_helpful_validation.append(1)
        for j in range(len(tf_idf_r_helpful_validation[i])):
            temp_helpful_validation.append(tf_idf_r_helpful_validation[i][j])
        new_helpful_validation.append(temp_helpful_validation)
        temp_helpful_validation=[]
X_validation = new_helpful_validation
y_validation = helpful_rate_validation

In [61]:
from sklearn import linear_model
regularization =[0.01,0.1,1,10,100,1000,10000,100000]
for k in regularization:
    clf = linear_model.Ridge(k, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_validation)
    s= 0
    for i in range(len(y_validation)):
        if predictions[i]<0:
            predictions[i]=0
        elif predictions[i]>1:
            predictions[i]=1
        s +=(y_validation[i]-predictions[i])**2
    MSE = s/len(y_validation)
    print(k,MSE)    

0.01 0.15291716986623757
0.1 0.15109803502480085
1 0.13755789403462637
10 0.10316849183571665
100 0.0764987379209909
1000 0.0847630702969212
10000 0.1773727099848295
100000 0.2682069253875025


In [91]:
    clf = linear_model.Ridge(100, fit_intercept=False)
    clf.fit(X_train, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_test)    

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.cross_validation import train_test_split

y1 = []
for i in y_train:
    if i>= 0.5:
        y1.append(1)
    else:
        y1.append(0)


vectorizer=TfidfVectorizer()
classifier=LogisticRegression()
classifier.fit(X_train,y1)
predictions=classifier.predict(X_train)

In [94]:
len(X_test)

7208